In [18]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
import matplotlib.pyplot as plt
from keras.layers import Dense,Activation
from keras.models import Sequential
from keras.models import Model

[2017-09-24 22:56:47,826] Making new env: CartPole-v0


In [20]:

gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


In [17]:
#Creating the agent

class agent():
    def __init__(self):
        self.lr = 1e-2
        self.s_size = 4
        self.a_size = 2
        self.h_size = 8
        
        self.model = Sequential()
        self.model.add(Dense(units = self.h_size, input_dim = self.s_size,kernel_initializer='random_normal',
                bias_initializer='zeros'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(units=self.a_size,kernel_initializer='random_normal',
                bias_initializer='zeros'))
        self.model.add(Activation('softmax'))
        self.model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])
        
    def query_model(self,s):
        #Keras expects each state observation to be a row (,4) Vector.
        #Gym is spitting it out as a (4,) (or column) vector. 
        #So lets reshape this shit.
        
        s = np.reshape(s,[1,4])
        
        #This above line does the following
        #new_s = np.zeros([1,4])
        #new_s[0,0] = s[0]
        #new_s[0,1] = s[1]
        #new_s[0,2] = s[2]
        #new_s[0,3] = s[3]
        #s = new_s
        
        return self.model.predict(s)
        
    def optimize_model(self,state,action,reward):
        
        #write code to optimize model
        return
        
        

In [ ]:
myAgent = agent()
total_episodes = 5000
max_ep = 99
update_frequency = 5

i = 0
total_reward = []
total_lenght = []

while i <total_episodes:
    s = env.reset()
    running_reward = 0
    ep_history = []
    
    for j in range(max_ep):
        #Probabilistically pick an action given our network outputs.
        a_dist = myAgent.query_model(s)
        a = np.random.choice(a_dist[0],p = a_dist[0])
        a = np.argmax(a_dist == a)
        
        s1,r,d,_ = env.step(a)
        ep_history.append([s,a,r,s1])
        
        s = s1
        running_reward = running_reward + r
        
        if d == True:
            #update network
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2])
            
            myAgent.optimize_model(ep_history[:,0],ep_history[:,1],ep_history[:,2])
            
            total_reward.append(running_reward)
            total_lenght.append(j)
            break
    
    if i%100 == 0:
        print(np.mean(total_reward[-100:]))
    i = i + 1
    
            
            